<a href="https://colab.research.google.com/github/ichekhovskikh/recommendation-system/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gensim
import os
import collections
import smart_open
import random

#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель с использованием Lee Background Corpus, включенного в gensim. Этот корпус содержит 314 документов, отобранных из службы почтовой рассылки Австралийской радиовещательной корпорации, которая предоставляет текстовые электронные письма с заголовками и охватывает ряд широких тем.

И мы проверим нашу модель на глаз, используя более короткий Lee Corpus, который содержит 50 документов.

In [0]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла (с латинской кодировкой), построчно считываем файл, предварительно обрабатываем каждую строку, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.) и возвращаем список слов. Обращаем внимание, что для данного файла (он же корпус) каждая непрерывная строка составляет один документ, а длина каждой строки (то есть документа) может варьироваться. Кроме того, для обучения модели нам нужно будет связать тег/номер с каждым документом учебного корпуса. В нашем случае тег - это просто номер строки, начинающийся с нуля.

In [0]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [0]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

Давайте посмотрим на учебный корпус:

In [20]:
train_corpus[:2]

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

Корпус тестирования выглядит так:

In [21]:
print(test_corpus[:2])

[['the', 'national', 'executive', 'of', 'the', 'strife', 'torn', 'democrats', 'last', 'night', 'appointed', 'little', 'known', 'west', 'australian', 'senator', 'brian', 'greig', 'as', 'interim', 'leader', 'shock', 'move', 'likely', 'to', 'provoke', 'further', 'conflict', 'between', 'the', 'party', 'senators', 'and', 'its', 'organisation', 'in', 'move', 'to', 'reassert', 'control', 'over', 'the', 'party', 'seven', 'senators', 'the', 'national', 'executive', 'last', 'night', 'rejected', 'aden', 'ridgeway', 'bid', 'to', 'become', 'interim', 'leader', 'in', 'favour', 'of', 'senator', 'greig', 'supporter', 'of', 'deposed', 'leader', 'natasha', 'stott', 'despoja', 'and', 'an', 'outspoken', 'gay', 'rights', 'activist'], ['cash', 'strapped', 'financial', 'services', 'group', 'amp', 'has', 'shelved', 'million', 'plan', 'to', 'buy', 'shares', 'back', 'from', 'investors', 'and', 'will', 'raise', 'million', 'in', 'fresh', 'capital', 'after', 'profits', 'crashed', 'in', 'the', 'six', 'months', 'to'

Обратите внимание, что корпус тестирования представляет собой просто список списков и не содержит никаких тегов.

# Обучение модели
## Создание объекта Doc2Vec
Теперь мы создадим модель Doc2Vec с векторным размером 50 слов и перебираем учебный корпус 40 раз. Мы устанавливаем минимальное количество слов на 2, чтобы отбрасывать слова с очень малым количеством вхождений. (Без множества репрезентативных примеров сохранение таких редких слов часто может ухудшить модель!) Типичные значения итераций в опубликованных результатах «Paragraph Vectors», использующих от десятков тысяч до миллионов документов, составляют 10–20. Больше итераций занимает больше времени и в конечном итоге достигает точки убывающей отдачи.

Однако это очень маленький набор данных (300 документов) с короткими документами (несколько сотен слов). Добавление пропусков может иногда помочь с такими маленькими наборами данных.

По сути, словарь - это словарь (доступный через model.wv.vocab) всех уникальных слов, извлеченных из учебного корпуса вместе со счетчиком (например, model.wv.vocab['penalty'].count для подсчета слова penalty).

In [22]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 6.35 s, sys: 191 ms, total: 6.55 s
Wall time: 3.59 s


## Inferring a Vector
Важно отметить, что теперь вы можете вывести вектор для любого фрагмента текста без необходимости переобучать модель, передав список слов в функцию model.infer_vector. Затем этот вектор можно сравнить с другими векторами по косинусному подобию.

In [23]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

array([-0.13778152,  0.10190844,  0.09070924, -0.1291998 ,  0.02204582,
       -0.04395645,  0.15486427, -0.15084767,  0.2042574 , -0.09058648,
       -0.11150775,  0.1230991 ,  0.07102051, -0.04443945,  0.17604499,
       -0.03153215,  0.04492974,  0.08625893,  0.14770153,  0.01397196,
       -0.01415225,  0.16656375,  0.11814791,  0.3571505 ,  0.089607  ,
        0.08551267,  0.04979853,  0.10949661, -0.06877135, -0.01836316,
       -0.00106802, -0.07862338, -0.01490802,  0.23243338,  0.15919799,
        0.19454263, -0.01167523,  0.28749686, -0.08449314, -0.00404665,
       -0.03478434,  0.12410981, -0.12244734,  0.13566194,  0.02238736,
        0.07469752,  0.16304158, -0.08340151, -0.15565938,  0.15476778],
      dtype=float32)

Обратите внимание, что infer_vector() принимает не строку, а список строковых токенов, которые уже должны были быть разбиты на токены так же, как свойство words исходных объектов обучающего документа.

Также обратите внимание, что, поскольку лежащие в основе алгоритмы обучения представляют собой проблему итеративной аппроксимации, в которой используется внутренняя рандомизация, повторные выводы одного и того же текста будут возвращать слегка разные векторы.

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом, а затем вернем ранг документа на основе самоподобия. По сути, мы притворяемся, что учебный корпус представляет собой новые невидимые данные, а затем видим, как они сравниваются с обученной моделью. Ожидается, что мы, вероятно, перегоним нашу модель (т.е. Все ранги будут меньше 2), и поэтому мы должны быть в состоянии найти похожие документы очень легко. Кроме того, мы будем отслеживать вторые ранги для сравнения менее похожих документов.

In [24]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Давайте посчитаем, как оценивается каждый документ по отношению к учебному корпусу.

In [25]:
collections.Counter(ranks)  # Results vary between runs due to random seeding and very small corpus

Counter({0: 292, 1: 8})

По сути, более 95% предполагаемых документов оказываются наиболее похожими на себя, и примерно в 5% случаев они по ошибке наиболее похожи на другие документы. проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя с пользой и согласованностью, хотя и не является реальным значением «точности».

Это здорово и не совсем удивительно. Мы можем взглянуть на пример:

In [26]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

Обратите внимание, что наиболее похожий документ (как правило, тот же текст) имеет показатель сходства, приближающийся к 1,0. Тем не менее, оценка сходства для документов второго ранга должна быть значительно ниже (при условии, что документы на самом деле разные), и рассуждение становится очевидным, когда мы исследуем сам текст.

Мы можем запустить следующую ячейку несколько раз, чтобы увидеть выборку других сравнений целевого документа.

In [27]:
# Pick a random document from the corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (272): «the storm clean up in sydney will resume in earnest this morning as fresh crews are brought in to replace state emergency service ses personnel who worked through the night the storm hit sydney early yesterday afternoon and two schoolgirls died when tree fell on them at reserve at hornsby heights in the city north number of other people were injured as the storm brought down trees and power poles and lifted roofs new south wales emergency services minister bob debus says welfare and emergency funding arrangements have been put in place with the declaration of natural disaster areas in campbeltown hornsby warringah and kurringai welfare services become available if they are needed local government is refunded any money it spends on the clean up or that it spends on repairing its own infrastructure low interest loans if they are needed are available to small business to help them get back on their feet again mr debus said energy australia says power has been restor

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [28]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (42): «pope john paul ii urged delegates at major summit on sustainable growth on sunday to pursue development that protects the environment and social justice in comments to tourists and the faithful at his summer residence southeast of rome the pope said god had put humans on earth to be his administrators of the land to cultivate it and take care of it in world ever more interdependent peace justice and the safekeeping of creation cannot but be the fruit of joint commitment of all in pursuing the common good john paul said»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (186, 0.7243496775627136): «united nationals secretary general kofi annan has accepted the nobel peace prize in the norwegian capital oslo declaring that to save one life is to save humanity itself mr annan told gala audience the world must respect the individual whose fundamental rights he says have been sacrificed too often for the good of the state the year old un chi

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
